Importiamo le librerie necessarie.

In [ ]:
import numpy as np
from scipy.signal import find_peaks
import matplotlib.pyplot as plt

Impariamo a leggere un file in formato CSV. Ci sono molte varianti di questo formato ma il più comune consiste di campi separati da ","


In [ ]:
spettro = np.genfromtxt('CO.csv', delimiter=',',skip_header=1,names=True)

Alternativamente possiamo usare read_csv dalla libreria pandas, che restituisce una tabella

In [ ]:
import pandas as pd
sp=pd.read_csv('CO.csv',skiprows=1)
sp

Tradizionalmente gli spettri IR hanno l'asse delle ascisse invertito, riportiamolo ad un ordine più adatto alla manipolazione. Inoltre per comodità estraiamo le variabili dalla tabella "spettro", la tabella ha le intestazioni 'cm1' e 'A'


In [ ]:
sp_x=spettro['cm1'][::-1] # invertiamo l'ordine del vettore con la notazione [::-1]
sp_y=spettro['A'][::-1]

Soffermiamoci ad osservare lo spettro per definire gli intervalli di interesse

In [ ]:
plt.plot(sp_x,sp_y)
plt.show()


Andiamo a selezionare una regione interessante dello spettro, in questo caso la banda per la transizione vibrazionale 0→1

In [ ]:
banda_01=(sp_x < 2300) & (sp_x > 2000)
banda_01

Abbiamo definito la condizione banda_01 che è un vettore di valori booleani. Adesso possiamo
usare la notazione comune a python, matlab, fortran ed altri linguaggi di alto livello che ci
consente di selezionare gli elementi di un vettore in corrispondenza dei valori True.

In [ ]:
x_01=sp_x[banda_01]
y_01=sp_y[banda_01]

plt.plot(x_01,y_01)
plt.show()


Adesso, sfruttando lo spazio per il codice sopra, provate a restringere ulteriormente l'intervallo di definizione dello spettro.


Passiamo all'analisi dello spettro. Troviamo i picchi sfruttando le capacità in scipy.signal.
La funzione find_peaks_cwt, non è stata disegnata per analizzare spettri IR, ma funziona comunque abbastanza bene per i nostri scopi.

In [ ]:
peakind, _=find_peaks(y_01, distance=20, height=0.2)
plt.plot(x_01,y_01)
plt.plot(x_01[peakind],y_01[peakind],'ob')
plt.show()


In [ ]:
find_peaks?

La funzione find_peaks richiede in ingresso lo spettro ed altri valori opzionali, che servono per trattare il rumore, la presenza di picchi minori e altro. Vengono restituiti gli indici corrispondenti ai massimi trovati nel vettore dello spettro, in questo caso y_01.
Abbiamo aggiunto al nostro grafico l'annotazione dei picchi trovati con cerchi blu 'ob'

Adesso andiamo a numerare i picchi, tramite l'indice m=J+1 per le transizioni J→J+1, e m=-J per le transizioni J→J-1. Cominciamo con il creare un indice m fittizio, per aggiustarlo successivamente. Ma prima eliminiamo alcuni picchi erroneamente trovati. Per semplicità andiamo solo a restringere lo spettro.

In [ ]:
m_01=np.arange(len(peakind))
p_01=x_01[peakind]
plt.plot(m_01,p_01,'o-r')
plt.show()


Ora bisogna aggiustare l'indice m. Possiamo farlo sistematicamente approfittando della discontinuità intorno ad m=0, visibile anche nel grafico. Intorno ad m=0 la distanza tra i picchi è doppia.


In [ ]:
def trova_00(x):
    d=x[1]-x[0]
    for i in range(len(x)-1):
        di=x[i+1]-x[i]
        if di>1.5*d:
            return 0.5*(x[i+1]+x[i])

def indice_m(x):
    "restituisce l'indice m per una serie di picchi"
    v00=trova_00(x)
    nsin=sum(x<v00)
    ndex=sum(x>v00)
    psin=np.arange(-nsin,0)
    pdex=np.arange(ndex)+1
    return np.append(psin, pdex)


In [ ]:
m_01=indice_m(p_01)
plt.plot(m_01,p_01,'o-r')
plt.axvline(0, ls='--')
m_01

Infine andiamo ad effettuare un fit polinomiale delle posizioni dei picchi in funzione di m. Il polinomio, che tiene conto della distorsione centrifuga e dell'accoppiamento rotovibrazionale, è di terzo grado. La libreria numpy consente questo calcolo tramite la funzione polyfit. 

In [ ]:
coeff = np.polyfit(m_01, p_01, 3)
polinomio=np.poly1d(coeff)
valori_fit=polinomio(m_01)


polyfit ritorna i coefficienti del polinomio. Usate il riquadro sopra per visionarli. Per calcolare i valori del polinomio è conveniente usare la funzione poly1d che permette di trasformare i coefficienti in un oggetto che può essere chiamato come una funzione. Provate per esempio a chiamare polinomio(x) con diversi valori di x.  

Verifichiamo il nostro fit. La grafica è la migliore prima opzione.

In [ ]:
plt.plot(m_01,p_01,'o-r')
plt.plot(m_01,valori_fit,'+b')
plt.show()

